In [1]:
import ipynb.fs.full.google_search as gs
import ipynb.fs.full.gather_hl_and_articles as reader
import ipynb.fs.full.open_ai_models as openaimodel
import ipynb.fs.full.data_operations as do
import pandas as pd
import pprint as pp
import json

In [2]:
def main():

    analysts = ['Sumeet Bagadia'#,
    #        'Vaishali Parekh', 
    #        'Ganesh Dongre',
    #        'Dharmesh Shah',
    #        'Seema Srivastava',
    #        'Anshul Jain',
    #        'Shiju Koothupalakkal', 
    #        'Sugandha Sachdeva',
    #        'Mahesh M Ojha'
                ]
    
    
    ops = do.gspread_Operator()
    srch = gs.google_Search()
    llm = openaimodel.OpenAIModels()
    rdr = reader.Livemint_reader()
    
    #for loop should begin here:
    for analyst in analysts:
        try:
            print(f'Starting mass link search for {analyst}...')
            link_ids_already_seen = ops.read_specific_article_columns(['link_id'])
            #print(link_ids_already_seen.shape)
            google_links = srch.search(query=analyst, terms='Buy', num_pages=10, silent_mode=False)
            print(f'Found {google_links.shape[0]} links for {analyst} from Google for Livemint. Proceeding to dedupe against known links...')
            ##remove links we have previously seen
            seen_indices = google_links[google_links['link_id'].isin(link_ids_already_seen['link_id'])].index.tolist()
            if(seen_indices):
                google_links.drop(index=seen_indices, inplace=True)
            print(f'Now remaining {google_links.shape[0]} links after dedupe...')
            response = llm.classify_headlines(google_links['title'], silent_mode=False)
            if response:
                output_json = json.loads(response)['output']
                classified_hl = pd.DataFrame(output_json)
                classified_titles = google_links.join(classified_hl, lsuffix='_orig', rsuffix='_copy')
                #ops.write_headlines_in_chunks(classified_titles)
                for idx, row in classified_titles:
                    article = rdr.read_article(row['link'])
                    if(article):
                        response2 = llm.gather_articles_susbtance(article, silent_mode=False)
                        if response2:
                            recos = pd.DataFrame(json.loads(response2)['output'])
                            ops.write_recommendations(recos)
                            ops.write_headlines(row)
                            break
                        else:
                            print(f'No recommendation could be gathered for {row['link']}...')
                    else:
                        print(f'No article could be retrieved for {row['link']}...') 
                    break
            else:
                print(f'No classifications could be done for the articles...')
        except Exception as e:
            print(f'Failed execution with exception: {e}')
        finally:
            continue
    
    return 0

In [4]:
if __name__ == '__main__':
    hl = main()

Starting mass link search for Sumeet Bagadia...
Running search for Sumeet+Bagadia on page: 1
Error: 429 at run page: 1
Combining all outcomes...
Deduping links...
Found 0 links for Sumeet Bagadia from Google for Livemint. Proceeding to dedupe against known links...
Now remaining 0 links after dedupe...
Sending titles for classification to OpenAI model...
Failed execution with exception: too many values to unpack (expected 2)
